In [1]:
from glob import glob
import os
import pandas as pd
from pivottablejs import pivot_ui

In [2]:
folder = 'Data/'

In [3]:
def read_l2(folder):
    folder_list = glob(folder+'*')
    folder_list = [f for f in folder_list if os.path.isdir(f)]
    file_list = [glob(folder + '/*') for folder in folder_list]
    df = pd.concat([pd.read_csv(f, converters={'code': str}, parse_dates=['datetime']) for file in file_list for f in file])
    if 'Unnamed: 0' in df.columns:
        del df['Unnamed: 0']
        
    df = df.assign(date=df['datetime'].apply(lambda x: x.strftime('%Y-%m-%d')))
    df['date'] = pd.to_datetime(df['date'])
    df.set_index(['code', 'date'], drop=False, inplace=True)
    df.sort_index(level=[0, 1], inplace=True)
    return df

In [4]:
df = read_l2(folder)

In [5]:
df = df.assign(buy_amount=df['buy_vol'] * df['price'])
df = df.assign(sell_amount=df['sell_vol'] * df['price'])

# 根据买单号计算总买价
df = df.set_index(['code', 'date', 'buy_no'], drop=False).join(
    pd.DataFrame(df.groupby(level=[0, 1]).apply(lambda x: x.groupby('buy_no')['buy_amount'].sum())) \
    .rename({'buy_amount': 'buy_amount_total'}, axis=1)
)

# 根据卖单号计算总卖价
df = df.set_index(['code', 'date', 'sell_no'], drop=False).join(
    pd.DataFrame(df.groupby(level=[0, 1]).apply(lambda x: x.groupby('sell_no')['sell_amount'].sum())) \
    .rename({'sell_amount': 'sell_amount_total'}, axis=1)
)

In [7]:
# 全部股票五天总买前5
df.groupby(level=[0, 1]) \
    .apply(lambda x: x.nlargest(5, ['buy_amount_total']))[['datetime', 'buy_no', 'buy_amount_total', 'sell_no', 'sell_amount_total']]

datetime   buy_no  \
code   date       code   date       sell_no                                
000078 2020-02-03 000078 2020-02-03 1987    2020-02-03 09:30:11     1991   
                                    1992    2020-02-03 09:30:11     1991   
                                    1993    2020-02-03 09:30:11     1991   
                                    1994    2020-02-03 09:30:11     1991   
                                    1995    2020-02-03 09:30:11     1991   
...                                                         ...      ...   
603880 2020-02-07 603880 2020-02-07 2473319 2020-02-07 11:12:14  2870970   
                                    2474644 2020-02-07 11:12:14  2870970   
                                    2474714 2020-02-07 11:12:14  2870970   
                                    2475258 2020-02-07 11:12:14  2870970   
                                    2475426 2020-02-07 11:12:14  2870970   

                                             buy_amount_total  sell_no  \
code   date       code   date       sell_no                              
000078 2020-02-03 000078 2020-02-03 1987         1.121901e+09     1987   
                                    1992         1.121901e+09     1992   
                                    1993         1.121901e+09     1993   
                                    1994         1.121901e+09     1994   
                                    1995         1.121901e+09     1995   
...                                                       ...      ...   
603880 2020-02-07 603880 2020-02-07 2473319      1.340278e+10  2473319   
                                    2474644      1.340278e+10  2474644   
                                    2474714      1.340278e+10  2474714   
                                    2475258      1.340278e+10  2475258   
                                    2475426      1.340278e+10  2475426   

                                             sell_amount_total  
code   date       code   date       sell_no                     
000078 2020-02-03 000078 2020-02-03 1987              503500.0  
                                    1992               53000.0  
                                    1993                2650.0  
                                    1994                5300.0  
                                    1995               10600.0  
...                                                        ...  
603880 2020-02-07 603880 2020-02-07 2473319            20910.0  
                                    2474644             6273.0  
                                    2474714            43911.0  
                                    2475258             2091.0  
                                    2475426             4182.0  

[800 rows x 5 columns]

In [9]:
# 全部股票五天总卖前5
df.groupby(level=[0, 1]) \
    .apply(lambda x: x.nlargest(5, ['sell_amount_total']))[['datetime', 'buy_no', 'buy_amount_total', 'sell_no', 'sell_amount_total']]

datetime  buy_no  \
code   date       code   date       sell_no                               
000078 2020-02-03 000078 2020-02-03 434     2020-02-03 09:25:00     432   
                                    434     2020-02-03 09:25:00     435   
                                    434     2020-02-03 09:25:00     436   
                                    434     2020-02-03 09:25:00     437   
                                    434     2020-02-03 09:25:00     438   
...                                                         ...     ...   
603880 2020-02-07 603880 2020-02-07 126324  2020-02-07 09:25:00   72536   
                                    126324  2020-02-07 09:25:00   72585   
                                    126324  2020-02-07 09:25:00   72813   
                                    126324  2020-02-07 09:25:00   73005   
                                    126324  2020-02-07 09:25:00   73025   

                                             buy_amount_total  sell_no  \
code   date       code   date       sell_no                              
000078 2020-02-03 000078 2020-02-03 434              127200.0      434   
                                    434                1590.0      434   
                                    434                 530.0      434   
                                    434              212000.0      434   
                                    434                1590.0      434   
...                                                       ...      ...   
603880 2020-02-07 603880 2020-02-07 126324          1731348.0   126324   
                                    126324             4182.0   126324   
                                    126324            23001.0   126324   
                                    126324            10455.0   126324   
                                    126324             2091.0   126324   

                                             sell_amount_total  
code   date       code   date       sell_no                     
000078 2020-02-03 000078 2020-02-03 434            736700000.0  
                                    434            736700000.0  
                                    434            736700000.0  
                                    434            736700000.0  
                                    434            736700000.0  
...                                                        ...  
603880 2020-02-07 603880 2020-02-07 126324         569421120.0  
                                    126324         569421120.0  
                                    126324         569421120.0  
                                    126324         569421120.0  
                                    126324         569421120.0  

[800 rows x 5 columns]

In [10]:
# 单只股票五天总买前20
df.loc['000078'].groupby(level=0).apply(lambda x: x.nlargest(20, ['buy_amount_total']))[['datetime', 'buy_amount_total', 'sell_amount_total']]

datetime  buy_amount_total  \
date       date       sell_no                                         
2020-02-03 2020-02-03 1987    2020-02-03 09:30:11      1.121901e+09   
                      1992    2020-02-03 09:30:11      1.121901e+09   
                      1993    2020-02-03 09:30:11      1.121901e+09   
                      1994    2020-02-03 09:30:11      1.121901e+09   
                      1995    2020-02-03 09:30:11      1.121901e+09   
...                                           ...               ...   
2020-02-07 2020-02-07 28085   2020-02-07 10:35:28      3.072960e+09   
                      28086   2020-02-07 10:35:28      3.072960e+09   
                      28087   2020-02-07 10:35:34      3.072960e+09   
                      28088   2020-02-07 10:35:34      3.072960e+09   
                      28089   2020-02-07 10:35:35      3.072960e+09   

                               sell_amount_total  
date       date       sell_no                     
2020-02-03 2020-02-03 1987              503500.0  
                      1992               53000.0  
                      1993                2650.0  
                      1994                5300.0  
                      1995               10600.0  
...                                          ...  
2020-02-07 2020-02-07 28085              27160.0  
                      28086               1552.0  
                      28087               7760.0  
                      28088                776.0  
                      28089                776.0  

[100 rows x 3 columns]

In [11]:
# 可视化
single_df = df.loc['000078'].groupby(level=0) \
    .apply(lambda x: x.nlargest(20, ['buy_amount_total']))[['datetime', 'buy_amount_total', 'sell_amount_total']]

In [12]:
pivot_ui(single_df)